In [ ]:
from unsloth import FastModel
import torch

In [ ]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    max_seq_length = 2048, 
    load_in_4bit = True,  
    load_in_8bit = False,
    full_finetuning = False, 
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           
    lora_alpha = 8,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_data_formats

dataset = load_dataset("csv", data_files="progressive_train_80.csv")["train"]
dataset = standardize_data_formats(dataset)


In [ ]:
dataset[120]

In [ ]:
def apply_chat_template(example):
    return {
        "text": tokenizer.apply_chat_template(
            [{"role": "user", "content": str(example["input"])},
             {"role": "assistant", "content": str(example["output"])}],
            tokenize=False
        )
    }

dataset = dataset.map(apply_chat_template)


In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,  
        warmup_steps = 0,
        num_train_epochs = 1,
        max_steps = 500,  
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    )
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[120]["input_ids"])

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[120]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
torch.cuda.empty_cache()  # Clear GPU memory

In [ ]:
trainer_stats = trainer.train()

In [ ]:
system_prompt1 = """
אתה תומך רגשי אמפתי ומכיל, המספק תמיכה רגשית באמצעות שיחות טקסט לאנשים במצוקה. מטרתך היא להקשיב באמת, לאפשר הבעת רגשות קשים – כולל מחשבות אובדניות – ולספק מרחב בטוח ולא שיפוטי.

- אפשר לשיח על מוות להיות נוכח. הפגן פתיחות ואי־בהלה: "הכאב נשמע בלתי נסבל... אתה חושב על לפגוע בעצמך?"
- נסה להשהות מחשבות על פעולה אובדנית: "אני איתך עכשיו. תן לי לעזור. לא חייבים להחליט עכשיו."
- אשר רגשות ואל תבטל אותם. הימנע מתגובות כמו "יהיה בסדר" או "זה יעבור".
- שאל שאלות פתוחות שמזמינות שיתוף: "מתי התחלת להרגיש ככה?", "מה עזר לך בעבר?", "מה קורה לך בימים האחרונים?"
- הפגן נוכחות: "אני כאן איתך, עכשיו. שום דבר אחר לא חשוב לי כמוך."
- שיקף אמפתיה באופן מדויק: "נשמע שאת מרגישה מותשת, חסרת אונים ובלתי נראית. ספרי לי עוד, זה חשוב לי להבין אותך."
- דובב בעדינות. אל תלחץ, אל תנחש. התעניין באמת.
- נסה להבין את עומק המחשבות: "כבר חשבת איך? או מתי?", בלי להבהיל ובלי לשפוט.
- עודד שיתוף באחרים או קבלת עזרה מקצועית, כחלק מהשיח ולא כהוראה חיצונית: "רוצה שנחשוב יחד על מי אפשר לפנות אליו?"
- שמור על שפה פשוטה, חמה ומכילה. גם שתיקה או בלבול הם חלק מהשיחה.

לעולם אל תיתן אבחנות קליניות או עצות טיפוליות. תפקידך הוא ללוות, לא לפתור.
"""


## Example

In [ ]:
messages = [
    {
            "role": "system",
            "content": [{"type": "text", "text": system_prompt1}]},
    {
    "role": "user",
    "content": [{"type" : "text", "text" : "אני לא מסוגל לקום מהמיטה, רבתי עם ההורים שלי ואני ממש בדיכאון",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# Saving the model

In [ ]:
new_model_local = "Gemma-3-12B-it-FirstResponder_progressive_train_80_500steps"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)